In [1]:
import pandas as pd
import numpy as np
import string
from collections import Counter
from utils import *
from vehicle import Vehicle

import networkx as nx
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx


## Estrazione dati con un piccolo parser dei dati

In [2]:
path='Instances/N6.txt'

In [3]:
#support
count = 0
#first 3 lines of the txt file
number_of_customers=0
one=0
number_of_vehicles=0
#vettori di informazione
vect_cord=[] #lista/vettore di coppiadi coordinate
vect_domanda=[] #linehaul
vect_carico=[] #backhaul
vec_b=[] #vettore delle istanze backhaul
vec_l=[] #vettore delle istanze linehaul
with open(path) as file:
    for line in file:
        #print(line)
        count+=1
        if count  == 1:
            number_of_customers=int((line.split())[0])
        if count  == 2:
            one=int((line.split())[0])
        if count == 3:
            number_of_vehicles=int((line.split())[0])
        if count > 3:
            vect_cord.append((int((line.split())[0]),int((line.split())[1])))
            vect_domanda.append((int((line.split())[2])))
            vect_carico.append((int((line.split())[3]))) 
            if int((line.split())[3]) == 0:
                vec_l.append(count-4) #il numero delle istanze linehaul
            if int((line.split())[3]) > 0:
                vec_b.append(count-4) # il numero delle istanze backhaul

#capacità massima di ogni mezzo
capacity=vect_carico[0]                     

In [4]:
df_data = pd.DataFrame({'': [number_of_customers, number_of_vehicles, capacity]},
                       index=['Number of customers', 'Number of Vehicles', 'Capacity'])

In [5]:
df_instances = pd.DataFrame({'Cordinate X': [ x[0] for x in vect_cord],
                             'Cordinate Y': [ x[1] for x in vect_cord], 
                             'Linehaul': vect_domanda, 
                             'Backhaul' : vect_carico,
                            })

In [6]:
df_data

,
Number of customers,150
Number of Vehicles,8
Capacity,8500


all'indice 0 è riportato il magazzino. dove il campo Backhaul rappresenta la capacità massima

In [7]:
df_instances

,Cordinate X,Cordinate Y,Linehaul,Backhaul
0,12000,16000,0,8500
1,1725,21949,0,820
2,22443,15651,0,291
3,11622,7533,0,344
4,14613,7181,0,244
5,771,10569,0,714
6,18508,15099,0,311
7,16029,5701,0,267
8,20666,29528,0,638
9,7898,29590,0,362


In [8]:
dict_coord = {}
for index, value in enumerate(vect_cord):
    dict_coord[index]=(value[0]/10000, value[1]/10000)

In [9]:
dict_coord

{0: (1.2, 1.6),
 1: (0.1725, 2.1949),
 2: (2.2443, 1.5651),
 3: (1.1622, 0.7533),
 4: (1.4613, 0.7181),
 5: (0.0771, 1.0569),
 6: (1.8508, 1.5099),
 7: (1.6029, 0.5701),
 8: (2.0666, 2.9528),
 9: (0.7898, 2.959),
 10: (0.7557, 1.1209),
 11: (2.2533, 2.7056),
 12: (1.3861, 1.0795),
 13: (1.0578, 0.3881),
 14: (0.1625, 1.9201),
 15: (1.8608, 2.6115),
 16: (0.567, 2.6554),
 17: (2.1409, 0.9013),
 18: (1.2465, 2.3593),
 19: (1.6852, 1.8097),
 20: (0.841, 0.1322),
 21: (0.3863, 2.9503),
 22: (0.4114, 0.9954),
 23: (0.4736, 1.8223),
 24: (0.9835, 1.0318),
 25: (1.8257, 2.0218),
 26: (1.2606, 2.3154),
 27: (1.8578, 1.136),
 28: (2.2262, 0.8762),
 29: (0.4626, 2.673),
 30: (0.1976, 1.806),
 31: (2.2413, 1.2376),
 32: (0.9725, 1.1171),
 33: (2.2724, 0.3555),
 34: (0.826, 0.954),
 35: (0.3994, 0.614),
 36: (0.7979, 0.3116),
 37: (1.2867, 2.5914),
 38: (1.9211, 1.2464),
 39: (2.3637, 0.8223),
 40: (0.5058, 2.8762),
 41: (1.0809, 1.8674),
 42: (2.3427, 1.2401),
 43: (2.0817, 0.3141),
 44: (1.2997,

In [10]:
import networkx as nx
from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral4
from bokeh.io import output_notebook
G=nx.Graph()
for e in list(dict_coord.keys()):
    G.add_node(e,pos=dict_coord[e])

In [11]:
pos=nx.get_node_attributes(G,'pos')

In [12]:
plot = Plot(plot_width=500, plot_height=500,
            x_range=Range1d(-1.3, 3.3), y_range=Range1d(-1.3, 3.3))
plot.title.text = "Graph Interaction"

node_hover_tool = HoverTool(tooltips=[("Magazzino", "@index"),("Coordinate", '@pos')])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

graph_renderer = from_networkx(G, pos, scale=1, center=(0, 0))
graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine( line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

output_notebook()
show(plot)

Loading BokehJS ...

## Calcolo dei Dati raccolti

In [13]:
distance = distance_matrix(vect_cord)

In [14]:
import numpy as np
distance = np.triu(distance)
dict_distance={}
for c, i in enumerate(distance):
    for k,j in enumerate(i):
        if j != 0:
            dict_distance[(c,k)]=j

In [15]:
distance

array([[    0.        , 11872.91985992, 10448.83007805, ...,
        13835.22045361, 17415.3082373 , 15899.84292375],
       [    0.        ,     0.        , 21654.10649276, ...,
        12284.49921649, 19918.39222929, 22856.19673086],
       [    0.        ,     0.        ,     0.        , ...,
        18101.67298898, 24898.49491435, 20238.11327669],
       ...,
       [    0.        ,     0.        ,     0.        , ...,
            0.        , 29379.61990905, 29526.59157438],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,  7781.94885617],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ]])

In [16]:
dict_distance

{(0, 1): 11872.919859916516,
 (0, 2): 10448.8300780518,
 (0, 3): 8475.433499237664,
 (0, 4): 9197.963361527376,
 (0, 5): 12473.419819760738,
 (0, 6): 6570.073439467781,
 (0, 7): 11059.034406312334,
 (0, 8): 16065.688282796975,
 (0, 9): 14195.580438995794,
 (0, 10): 6534.059228381696,
 (0, 11): 15270.207104031038,
 (0, 12): 5527.689028879971,
 (0, 13): 12202.141000660498,
 (0, 14): 10857.579196119179,
 (0, 15): 12082.172362617577,
 (0, 16): 12306.738641898592,
 (0, 17): 11719.532840518857,
 (0, 18): 7607.225118267502,
 (0, 19): 5285.765129099098,
 (0, 20): 15110.651342678779,
 (0, 21): 15765.207832439128,
 (0, 22): 9936.956878239936,
 (0, 23): 7596.540330966459,
 (0, 24): 6080.489207292453,
 (0, 25): 7545.964020587429,
 (0, 26): 7179.620602789538,
 (0, 27): 8049.825091267511,
 (0, 28): 12557.758080167017,
 (0, 29): 13019.553602178532,
 (0, 30): 10233.483082509103,
 (0, 31): 11025.604065084144,
 (0, 32): 5338.058261203225,
 (0, 33): 16428.091824676412,
 (0, 34): 7464.529456034051,
 (0, 3

In [17]:
from collections import Counter
c = Counter(dict_distance) #inizializzo una variabile della classe Counter 
dict_distance_Order=c.most_common() #ordino per i valori dei saving in maniera decrescente
dict_distance = {} # svuoto il vecchio dizionario per questioni di memoria
for e in dict_distance_Order: # ciclo su ogni valore del vettore della classe counter che è composto da copie 
    if 0 in e[0]:
        dict_distance[e[0]]=e[1] # reinserisco su K il nome delle istanze e su V il valore a loro associato

In [18]:
dict_distance
vect_b_new= (vec_b.copy())[1:]

In [19]:
vect_b_new

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50]

In [20]:
lista_k= list(dict_distance.keys())
for e in lista_k:
    for i in vect_b_new:
        if i in e:
            dict_distance.pop(e)

In [21]:
dict_distance

{(0, 117): 18551.330868700497,
 (0, 149): 17415.308237295143,
 (0, 150): 15899.842923752423,
 (0, 144): 15428.990407670879,
 (0, 85): 15416.36176275064,
 (0, 64): 15284.640820117429,
 (0, 93): 14670.085207659838,
 (0, 113): 14589.004935224335,
 (0, 77): 14488.33920779052,
 (0, 88): 14415.519310798345,
 (0, 68): 14187.13008328323,
 (0, 79): 14029.025803668621,
 (0, 131): 13979.817523844866,
 (0, 148): 13835.220453610416,
 (0, 69): 13807.044940898832,
 (0, 53): 13765.693916399565,
 (0, 121): 13763.725694738325,
 (0, 61): 13559.865080449732,
 (0, 71): 13343.034175179197,
 (0, 118): 12961.833280828758,
 (0, 95): 12793.467864500228,
 (0, 86): 12738.282341037979,
 (0, 114): 12640.306523182102,
 (0, 137): 12484.200294772589,
 (0, 92): 12434.504131649159,
 (0, 70): 12282.710816428107,
 (0, 134): 12135.840473572483,
 (0, 147): 12101.076852908587,
 (0, 75): 11932.132248680451,
 (0, 63): 11872.380132054397,
 (0, 99): 11870.548470900576,
 (0, 123): 11847.526746118787,
 (0, 112): 11844.21622565208,

In [22]:
lista_k=list(dict_distance.keys())
dict_init={}
lista_k=(lista_k[-number_of_vehicles:])
lista_k.reverse()
for e in lista_k:
    dict_init[e]= dict_distance.get(e)

In [23]:
dict_init

{(0, 87): 227.6532450899833,
 (0, 62): 965.186510473494,
 (0, 105): 2044.9356958105063,
 (0, 55): 3749.877464664679,
 (0, 57): 3764.0573853223864,
 (0, 104): 3957.0102350132984,
 (0, 119): 4264.062968578208,
 (0, 83): 4381.0506730691895}

In [24]:
saving=saving_matrix(distance)

In [25]:
dict_Saving_Order=savingOrder(saving)

## Mappa dei saving ordinati

In [26]:
dict_b, dict_l , dict_merge= route_Linehaul_Backhaul(dict_Saving_Order, vec_b, vec_l)

In [27]:
dict_l

{(117, 149): 15710.476867335052,
 (64, 144): 15270.59217535751,
 (113, 117): 14221.017023261578,
 (68, 85): 14098.244165568962,
 (53, 85): 13650.144052500853,
 (64, 117): 13627.26872449476,
 (113, 144): 13594.427789497273,
 (113, 149): 13424.755393389929,
 (61, 77): 13265.6994407291,
 (64, 150): 13139.304932707924,
 (144, 150): 13090.05645499719,
 (53, 68): 13053.634163572884,
 (64, 113): 12764.546111353542,
 (144, 149): 12342.408869083705,
 (86, 88): 12247.186310542158,
 (64, 149): 12241.663293930746,
 (61, 118): 12231.98055595887,
 (93, 149): 12032.111696205453,
 (70, 79): 11968.88794420387,
 (63, 79): 11872.36890003281,
 (99, 147): 11837.558352609272,
 (147, 150): 11798.6430993108,
 (99, 150): 11639.228648791594,
 (120, 149): 11510.341090900507,
 (93, 117): 11463.539604844034,
 (77, 118): 11396.750215940507,
 (113, 150): 11367.360160409578,
 (53, 88): 11279.87232811678,
 (106, 122): 11227.742711340701,
 (63, 70): 11202.782578071503,
 (58, 146): 11197.710821403009,
 (146, 147): 11162

## Inizializzo la lista delle rotte

In [28]:
'''list_route = []
set_iniziale = {0}
set_first = {0}
set_visited = {0}
lista_keys= list(dict_l.keys())
for k in lista_keys:
    if len(list_route) < number_of_vehicles and k[0] not in set_iniziale and k[1] not in set_iniziale:
        list_route.append(Vehicle([0, k[0], k[1]], vect_domanda[k[0]]+vect_domanda[k[1]],
                         distance[0][k[0]]+distance[k[0]][k[1]]))
        set_iniziale.add(k[0])
        set_iniziale.add(k[1])
        set_first.add(k[0])
        set_visited.add(k[0])
        set_visited.add(k[1])'''

'list_route = []\nset_iniziale = {0}\nset_first = {0}\nset_visited = {0}\nlista_keys= list(dict_l.keys())\nfor k in lista_keys:\n    if len(list_route) < number_of_vehicles and k[0] not in set_iniziale and k[1] not in set_iniziale:\n        list_route.append(Vehicle([0, k[0], k[1]], vect_domanda[k[0]]+vect_domanda[k[1]],\n                         distance[0][k[0]]+distance[k[0]][k[1]]))\n        set_iniziale.add(k[0])\n        set_iniziale.add(k[1])\n        set_first.add(k[0])\n        set_visited.add(k[0])\n        set_visited.add(k[1])'

In [29]:
list_route = []
set_iniziale = {0}
set_first = {0}
set_visited = {0}
lista_keys= list(dict_init.keys())


In [30]:
lista_keys

[(0, 87), (0, 62), (0, 105), (0, 55), (0, 57), (0, 104), (0, 119), (0, 83)]

In [31]:
for k in lista_keys:
    print(k)
    if len(list_route) < number_of_vehicles and k[1] not in set_iniziale:
        list_route.append(Vehicle([k[0], k[1]], vect_domanda[k[1]], distance[0][k[1]]))
        set_iniziale.add(k[0])
        set_iniziale.add(k[1])
        set_first.add(k[0])
        set_visited.add(k[0])
        set_visited.add(k[1])
        

(0, 87)
(0, 62)
(0, 105)
(0, 55)
(0, 57)
(0, 104)
(0, 119)
(0, 83)


In [32]:
for e in list_route:
    e.delivery = e.weight
    print(f'Mezzo {list_route.index(e)+1}:\t Rotta: {e.route}\t\t Costo: {e.cost}\t Delivery: {e.delivery}\t Pickup: {e.pickUp}')

Mezzo 1:	 Rotta: [0, 87]		 Costo: 227.6532450899833	 Delivery: 498	 Pickup: 0
Mezzo 2:	 Rotta: [0, 62]		 Costo: 965.186510473494	 Delivery: 720	 Pickup: 0
Mezzo 3:	 Rotta: [0, 105]		 Costo: 2044.9356958105063	 Delivery: 534	 Pickup: 0
Mezzo 4:	 Rotta: [0, 55]		 Costo: 3749.877464664679	 Delivery: 187	 Pickup: 0
Mezzo 5:	 Rotta: [0, 57]		 Costo: 3764.0573853223864	 Delivery: 195	 Pickup: 0
Mezzo 6:	 Rotta: [0, 104]		 Costo: 3957.0102350132984	 Delivery: 840	 Pickup: 0
Mezzo 7:	 Rotta: [0, 119]		 Costo: 4264.062968578208	 Delivery: 805	 Pickup: 0
Mezzo 8:	 Rotta: [0, 83]		 Costo: 4381.0506730691895	 Delivery: 545	 Pickup: 0


In [33]:
#remove_first_instance(dict_l, set_first)

In [34]:
c = 0
temp = dict_l.copy()
set_first = {0}
i = 0 
while(i < np.math.factorial(number_of_customers)/(np.math.factorial(number_of_customers-2)*2)):
    lista_keys = list(dict_l.keys())
    flag = True
    for k in lista_keys:
        #if 17 in k and list_route[c].route[-1] in k: #controllo cosa c'è quando arriva a 17
            #print(f'La route {c}: \t\t {list_route[c].route} \t\t capacità raggiunta: {list_route[c].weight} \t\t pair {k}')
            #print('----')
        if list_route[c].route[-1] in k \
        and different(list_route[c].route[-1], k) not in set_visited \
        and list_route[c].weight + vect_domanda[different(list_route[c].route[-1],k)] <= capacity:
                
                list_route[c].add_weight(vect_domanda[different(list_route[c].route[-1],k)])
                list_route[c].add_route(k)           
                set_visited.add(list_route[c].route[-1])
                list_route[c].add_cost(distance[k[0]][k[1]])
                set_first.add(k[0])

                break

    #remove_first_instance(dict_l, set_first)
    c = (c + 1) % number_of_vehicles
    
    i += 1

In [35]:
len(dict_l)

4950

In [36]:
for e in list_route:
    e.delivery = e.weight
    e.weight = 0
    print(f'Mezzo {list_route.index(e)+1}:\t Rotta: {e.route}\t\t Costo: {e.cost}\t Delivery: {e.delivery}\t Pickup: {e.pickUp}')

Mezzo 1:	 Rotta: [0, 87, 100, 91, 95, 69, 134, 103, 65, 73, 92, 133, 56, 74]		 Costo: 34669.42101713529	 Delivery: 6445	 Pickup: 0
Mezzo 2:	 Rotta: [0, 62, 78, 114, 123, 102, 96, 59, 112, 111, 81, 145, 127, 51]		 Costo: 65907.63067629254	 Delivery: 8436	 Pickup: 0
Mezzo 3:	 Rotta: [0, 105, 132, 136, 149, 117, 113, 144, 126, 116, 130, 128, 143, 107]		 Costo: 56177.655997851485	 Delivery: 5839	 Pickup: 0
Mezzo 4:	 Rotta: [0, 55, 71, 148, 121, 115, 101, 94, 80, 137, 106, 67, 93, 72]		 Costo: 72952.96933420967	 Delivery: 6877	 Pickup: 0
Mezzo 5:	 Rotta: [0, 57, 86, 88, 53, 85, 68, 90, 138, 52, 122, 66, 110]		 Costo: 46887.89220345518	 Delivery: 6341	 Pickup: 0
Mezzo 6:	 Rotta: [0, 104, 146, 58, 147, 99, 150, 64, 120, 97, 98, 75, 124]		 Costo: 38089.27522873957	 Delivery: 6040	 Pickup: 0
Mezzo 7:	 Rotta: [0, 119, 142, 76, 79, 70, 63, 129, 82, 89, 140, 139, 84]		 Costo: 33872.59803310613	 Delivery: 6373	 Pickup: 0
Mezzo 8:	 Rotta: [0, 83, 131, 61, 77, 118, 108, 109, 54, 135, 141, 125, 60]		 

In [37]:
dict_merge

{(20, 150): 14647.775433558703,
 (20, 144): 13654.87288056057,
 (21, 85): 13631.543564725978,
 (50, 150): 13621.64144012548,
 (20, 64): 13602.915218242973,
 (36, 64): 13371.246976707243,
 (36, 144): 13357.7275078921,
 (39, 69): 13134.800937578608,
 (45, 68): 12998.91798100519,
 (29, 68): 12987.12174997883,
 (45, 85): 12922.698994852783,
 (40, 85): 12913.940135606244,
 (29, 85): 12906.547028552117,
 (36, 150): 12876.227538158548,
 (35, 149): 12659.77283105538,
 (11, 131): 12564.727067372207,
 (9, 71): 12452.144170745429,
 (35, 117): 12450.609190079189,
 (40, 68): 12371.477032725386,
 (45, 53): 12326.215886001046,
 (29, 53): 12283.477670367967,
 (28, 69): 12242.382037906624,
 (21, 68): 12205.526187445315,
 (8, 77): 12145.670430519605,
 (13, 150): 12107.821826764697,
 (36, 113): 12088.28783807,
 (20, 117): 12072.597582015514,
 (28, 95): 12016.105658061042,
 (16, 68): 12015.460679345328,
 (36, 117): 11974.473868582583,
 (42, 92): 11955.875883423043,
 (16, 85): 11930.551383513122,
 (15, 61)

In [38]:
for e in dict_merge:
    if 17 in e:
        print(e)

(17, 95)
(17, 69)
(17, 134)
(17, 103)
(17, 92)
(17, 65)
(17, 133)
(17, 73)
(17, 91)
(17, 100)
(17, 70)
(17, 79)
(17, 76)
(17, 56)
(17, 63)
(17, 74)
(17, 107)
(17, 142)
(17, 51)
(17, 146)
(17, 72)
(17, 129)
(17, 150)
(17, 114)
(17, 58)
(17, 147)
(17, 99)
(17, 78)
(17, 60)
(17, 82)
(17, 64)
(17, 144)
(17, 123)
(17, 116)
(17, 113)
(17, 117)
(17, 89)
(17, 131)
(17, 149)
(17, 126)
(17, 102)
(17, 119)
(17, 120)
(17, 96)
(17, 61)
(17, 140)
(17, 77)
(17, 118)
(17, 104)
(17, 139)
(17, 97)
(17, 136)
(17, 93)
(17, 143)
(17, 130)
(17, 98)
(17, 83)
(17, 128)
(17, 121)
(17, 59)
(17, 75)
(17, 66)
(17, 127)
(17, 67)
(17, 110)
(17, 115)
(17, 84)
(17, 124)
(17, 108)
(17, 148)
(17, 109)
(17, 101)
(17, 132)
(17, 62)
(17, 94)
(17, 71)
(17, 54)
(17, 145)
(17, 112)
(17, 135)
(17, 122)
(17, 111)
(17, 55)
(17, 106)
(17, 80)
(17, 125)
(17, 105)
(17, 90)
(17, 87)
(17, 141)
(17, 138)
(17, 68)
(17, 85)
(17, 53)
(17, 137)
(17, 81)
(17, 86)
(17, 57)
(17, 52)
(17, 88)


In [39]:
a=vect_carico[4]+vect_carico[5]
b=vect_carico[3]
c=vect_carico[2]
d=vect_carico[1]

In [40]:
print(a)
print(b)
print(c)
print(d)

958
344
291
820


In [41]:
c = 0

while(c < len(list_route)):
    if len(set_visited) <= number_of_customers:
        merge = list(dict_merge.keys())
        back = list(dict_b.keys())
        for k in merge:
            if list_route[c].route[-1] in k and different(list_route[c].route[-1], k) not in set_visited:
                    list_route[c].add_weight(vect_carico[different(list_route[c].route[-1],k)])
                    list_route[c].add_route(k)
                    set_visited.add(list_route[c].route[-1])
                    list_route[c].add_cost(distance[k[0]][k[1]]) 
                    break
    c += 1

In [42]:

c = 0
temp = dict_b.copy()
set_first = {0}
i = 0 
while(i < np.math.factorial(number_of_customers)/(np.math.factorial(number_of_customers-2)*2)):
    lista_keys = list(dict_b.keys())
    flag = True
    for k in lista_keys:
        if list_route[c].route[-1] in k \
        and different(list_route[c].route[-1], k) not in set_visited \
        and list_route[c].weight + vect_carico[different(list_route[c].route[-1],k)] <= capacity:
                
                list_route[c].add_weight(vect_carico[different(list_route[c].route[-1],k)])
                list_route[c].add_route(k)           
                set_visited.add(list_route[c].route[-1])
                list_route[c].add_cost(distance[k[0]][k[1]])
                set_first.add(k[0])
                break

    #remove_first_instance(dict_l, set_first)
    c = (c + 1) % number_of_vehicles
    
    i += 1

In [43]:
for e in list_route:
    e.route.append(0)
    e.cost += distance[0][e.route[-1]]
    e.pickUp= e.weight

In [44]:
for e in list_route:
    print(e.route)

[0, 87, 100, 91, 95, 69, 134, 103, 65, 73, 92, 133, 56, 74, 27, 28, 17, 7, 11, 15, 46, 0]
[0, 62, 78, 114, 123, 102, 96, 59, 112, 111, 81, 145, 127, 51, 38, 39, 43, 4, 25, 18, 41, 0]
[0, 105, 132, 136, 149, 117, 113, 144, 126, 116, 130, 128, 143, 107, 12, 48, 33, 3, 14, 21, 0]
[0, 55, 71, 148, 121, 115, 101, 94, 80, 137, 106, 67, 93, 72, 6, 42, 31, 2, 8, 9, 0]
[0, 57, 86, 88, 53, 85, 68, 90, 138, 52, 122, 66, 110, 10, 35, 22, 5, 1, 29, 0]
[0, 104, 146, 58, 147, 99, 150, 64, 120, 97, 98, 75, 124, 32, 49, 36, 34, 23, 40, 0]
[0, 119, 142, 76, 79, 70, 63, 129, 82, 89, 140, 139, 84, 24, 47, 13, 30, 45, 16, 0]
[0, 83, 131, 61, 77, 118, 108, 109, 54, 135, 141, 125, 60, 44, 50, 20, 19, 37, 26, 0]


In [45]:
for e in list_route:
    print(f'Mezzo {list_route.index(e)+1}:\t Rotta: {e.route}\t\t Costo: {e.cost}\t Delivery: {e.delivery}\t Pickup: {e.pickUp}')

Mezzo 1:	 Rotta: [0, 87, 100, 91, 95, 69, 134, 103, 65, 73, 92, 133, 56, 74, 27, 28, 17, 7, 11, 15, 46, 0]		 Costo: 83919.96643697422	 Delivery: 6445	 Pickup: 3584
Mezzo 2:	 Rotta: [0, 62, 78, 114, 123, 102, 96, 59, 112, 111, 81, 145, 127, 51, 38, 39, 43, 4, 25, 18, 41, 0]		 Costo: 113082.0287613702	 Delivery: 8436	 Pickup: 3495
Mezzo 3:	 Rotta: [0, 105, 132, 136, 149, 117, 113, 144, 126, 116, 130, 128, 143, 107, 12, 48, 33, 3, 14, 21, 0]		 Costo: 109684.54061172307	 Delivery: 5839	 Pickup: 2651
Mezzo 4:	 Rotta: [0, 55, 71, 148, 121, 115, 101, 94, 80, 137, 106, 67, 93, 72, 6, 42, 31, 2, 8, 9, 0]		 Costo: 110314.09195544149	 Delivery: 6877	 Pickup: 2502
Mezzo 5:	 Rotta: [0, 57, 86, 88, 53, 85, 68, 90, 138, 52, 122, 66, 110, 10, 35, 22, 5, 1, 29, 0]		 Costo: 79350.88971014105	 Delivery: 6341	 Pickup: 3266
Mezzo 6:	 Rotta: [0, 104, 146, 58, 147, 99, 150, 64, 120, 97, 98, 75, 124, 32, 49, 36, 34, 23, 40, 0]		 Costo: 77016.67390337637	 Delivery: 6040	 Pickup: 2626
Mezzo 7:	 Rotta: [0, 119, 

In [46]:
print(f'numero nodi visitati, compreso il magazzino: {len(set_visited)}\nlista dei nodi: {set_visited}')

numero nodi visitati, compreso il magazzino: 151
lista dei nodi: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150}


In [47]:
tot_cost=sum([x.cost for x in list_route])

In [48]:
path_solutions ='RPA_Solutions/Detailed_Solution_N6.txt'

round(((tot_cost * 100) / extract_total_cost_BP(path_solutions))-100 , 2)


98.53

In [49]:
list_route[3].route

[0,
 55,
 71,
 148,
 121,
 115,
 101,
 94,
 80,
 137,
 106,
 67,
 93,
 72,
 6,
 42,
 31,
 2,
 8,
 9,
 0]

In [50]:
for i in range(len(list_route[3].route)-1):
    G.add_edge(list_route[3].route[i],list_route[3].route[i+1])

In [51]:
plot = Plot(plot_width=500, plot_height=500,
            x_range=Range1d(-1.3, 3.3), y_range=Range1d(-1.3, 3.3))
plot.title.text = "Graph Interaction"

node_hover_tool = HoverTool(tooltips=[("Magazzino", "@index"),("Coordinate", '@pos')])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

graph_renderer = from_networkx(G, pos, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine( line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)



output_notebook()
show(plot)

Loading BokehJS ...